In [ ]:
from nbdev import *
%nbdev_default_export hierarchy

Cells will be exported to pct.hierarchy,
unless a different module is specified after an export flag: `%nbdev_export special.module`


[autoreload of pct.functions failed: Traceback (most recent call last):
  File "C:\Users\ruper\.conda\envs\pctdev\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\ruper\.conda\envs\pctdev\lib\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "C:\Users\ruper\.conda\envs\pctdev\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "C:\Users\ruper\.conda\envs\pctdev\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "..\pct\functions.py", line 9, in <module>
    class Proportional(BaseFunction):
NameError: name 'BaseFunction' is not defined
]


In [ ]:
%nbdev_hide
%reload_ext autoreload
%autoreload 2

In [ ]:
# hide
import sys
sys.path.append("..")

# Hierarchy

Creation of Perceptual Control hierarchies.

In [ ]:
%nbdev_export
import numpy as np
from pct.nodes import PCTNode
from pct.functions import Variable

In [ ]:
%nbdev_export
class PCTHierarchy():
    "A hierarchical perceptual control system, of PCTNodes."
    def __init__(self, rows=1, cols=1, name="pcthierarchy", links="sparse", history=False, **pargs):
        self.links_built = False
        self.name=name 
        
        self.hierarchy = []
        print(self.hierarchy)
        for r in range(rows):
            col_list=[]
            for c in range(cols):
                node = PCTNode(name=f'row{r}col{c}')
                self.handle_links(node, r, c, links)
                col_list.append(node)
                
            self.hierarchy.append(col_list)
                
#        if history:
#            self.history = PCTNodeData()

    def handle_links(self, node, row, col, links_type):
        if row == 0 or links_type == None:
            return
        
        if links_type == "sparse":
            node.add_link("perception", self.hierarchy[row-1][col])
        
        if links_type == "dense":
            for column in range(len(self.hierarchy[row])):
                node.add_link("perception", self.hierarchy[row-1][column])
            
            
            
            
    def get_config(self):
        config = {"type": type(self).__name__,
                    "name": self.name}        
        
        levels = {}
        for row in range(len(self.hierarchy)):
            level ={'level':row}
            columns={}
            for col in range(len(self.hierarchy[row])):
                column={'col':col}
                nodeconfig = self.hierarchy[row][col].get_config()
                #print(nodeconfig)
                column['node']=nodeconfig
                #print(column)
                columns[f'col{col}']=column
            level['nodes']=columns
            levels[f'level{row}']=level
        config['levels']=levels
        return config       


In [ ]:
hpct = PCTHierarchy(2,2)
hpct.hierarchy


[]


[[<pct.nodes.PCTNode at 0x2abffe51088>, <pct.nodes.PCTNode at 0x2abffdb70c8>],
 [<pct.nodes.PCTNode at 0x2abffdb7a88>, <pct.nodes.PCTNode at 0x2abffdb7d88>]]

In [ ]:
#hpct.get_config()

AttributeError: 'PCTNode' object has no attribute 'get_name'

In [ ]:
#hide
from nbdev import *
notebook2script()